In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from pandas import Timestamp
import csv
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pydit import file_tools
tools= file_tools.FileTools()

In [2]:
custom_date_parser=lambda dates: [ datetime.strptime(str(d), "%Y-%m-%d") for d in dates]  
# for datetime "%Y-%m-%d %H:%M:%S"
dfraw = pd.read_csv('data.csv', quoting=csv.QUOTE_NONNUMERIC,quotechar='"',sep = ',', header = 0, index_col = None)
dfraw["date_trans"]=pd.to_datetime(dfraw["date_trans"],format="%Y-%m-%d")
dfraw

,id,ref,date_trans,status,amount,stage,status_code
0,1.0,INV-220001,2022-01-01,OPEN,35.94,PAID,1.0
1,2.0,INV-220002,2022-01-02,CLOSED,99.99,FULF,4.0
2,3.0,INV-220003,2022-01-03,CANCELLED,13.00,CANCELLED,99.0
3,4.0,INV-220003,2022-01-04,OPEN,NaN,CART,1.0
4,5.0,INV-220005,2022-01-04,CLOSED,204.20,FULF,4.0
5,6.0,INV-220006,2022-01-15,OPEN,-4.20,1,NaN
6,7.0,NaN,2022-01-06,NaN,0.00,NaN,NaN
7,8.0,INV-220007,2022-01-15,PENDING,50.40,ORDER,2.0
8,9.0,NaN,NaT,ERROR,0.00,NaN,NaN
9,10.0,INV-220007,2022-01-15,PENDING,50.40,ORDER,2.0


In [3]:
tools.config()

{'temp_path': './',
 'output_path': './',
 'input_path': './',
 'max_rows_to_excel': 200000}

In [4]:
tools.profile_dataframe(dfraw)

,column,dtype,records,count_unique,nans,value_counts,max,min,sum,sum_abs,std,empty_strings
0,id,float64,15,15,0,[],15,1,120.0,120.0,4.472136,NaN
1,ref,object,15,11,2,[],220012,220001,NaN,NaN,NaN,0.0
2,date_trans,datetime64[ns],15,8,1,[],2022-01-17 00:00:00,2022-01-01 00:00:00,NaN,NaN,NaN,NaN
3,status,object,15,5,1,[],NaN,NaN,NaN,NaN,NaN,0.0
4,amount,float64,15,7,1,[],204.2,-4.2,NaN,NaN,52.463812,NaN
5,stage,object,15,6,2,[],1,1,NaN,NaN,NaN,0.0
6,status_code,float64,15,4,3,"{2.0: 6, nan: 3, 4.0: 3, 1.0: 2, 99.0: 1}",99,1,NaN,NaN,27.916949,NaN


In [5]:
type(dfraw[['status',"stage"]])

pandas.core.frame.DataFrame

In [6]:
def group_categories(df_in, cols, top_n_values_to_keep=0, translation_dict={}):
    df=df_in.copy()
    if isinstance(cols,str):
        if not cols in df.columns:
            return "Not found"
        else:
            col=cols
    else:
        if not isinstance(cols,list):
            return "Please provide a list or a string"
        
        check=  all(item in df.columns  for item in cols)
        if not check:
            return "not all columns provided are in the dataframe, check for typos"
        if len(cols)==1:
            col=cols[0]
        elif len(cols)>1:
            def concat_categories(r,cols):              
                try:
                    v="_".join([str(v) for v in r[cols].values])
                except:
                    v=np.NAN
                return v    
            col="_".join(cols)
            df[col]=df.apply(lambda r: concat_categories(r,cols), axis=1)
            
        else:
            return "empty list"
        
    if translation_dict:
            df[col+"_translate"]=df.apply(lambda r: translation_dict[r[col]] if r[col] in translation_dict else "OTHER" ,axis=1) 
            col=col+"_translate" 
        
        
    if top_n_values_to_keep>0:
        print(col)
        print(df[col].value_counts())
        value_counts=df[col].value_counts().reset_index()
        value_counts_topN=list(value_counts[0:top_n_values_to_keep]['index'])
        df[col+"_collapsed"]=df.apply(lambda r: str.strip(str.upper(str(r[col]))) if r[col] in value_counts_topN else "OTHER" ,axis=1)
        
        print(value_counts_topN)
    
    
    return df
df=group_categories(dfraw, ["status"],2,{"OPEN":"OPEN","PENDING":"OPEN","CLOSED":"CLOSED"})

status_translate
OPEN      9
OTHER     3
CLOSED    3
Name: status_translate, dtype: int64
['OPEN', 'OTHER']


In [ ]:
def print_red(*args):
    print("\x1b[31m"+" ".join([str(x) for x in args])+"\x1b[0m")

def print_green(*args):
    print("\x1b[32m"+" ".join([str(x) for x in args])+"\x1b[0m")


In [ ]:
print_red("hello world",", this should be going red")

In [8]:
def cardinality_analysis(df_in):
    df=df_in.copy()
    count_unique= df.nunique().to_list()
    count_nulls=df.isnull().sum().to_list()
    card_perc= [i[0]/i[1] for i in zip(count_unique, df.count())] #zip just joins similar lists together  
    cardinality=list(zip(df.columns,count_unique, df.count(),count_nulls,card_perc))
    cardinality_df=pd.DataFrame(cardinality, columns=["name","cardinality","count","count_nulls","cardinality_percent"])
    return cardinality_df
cardinality_analysis(dfraw)

,name,cardinality,count,count_nulls,cardinality_percent
0,id,15,15,0,1.000000
1,ref,11,13,2,0.846154
2,date_trans,8,14,1,0.571429
3,status,5,14,1,0.357143
4,amount,7,14,1,0.500000
5,stage,6,13,2,0.461538
6,status_code,4,12,3,0.333333
